In [1]:
!pip install -q kaggle
import json

#Create Credential
######################## Insert Your username and Key ######################
data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
######################## Insert Your username and Key ######################
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

In [2]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c brain-motor-imagery-classification

ref                                                             title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows                          Netflix Movies and TV Shows                        1MB  2024-04-10 09:48:38          18347        399  1.0              
ashishkumarak/netflix-reviews-playstore-daily-updated           Netflix Reviews [DAILY UPDATED]                   12MB  2024-05-14 23:37:32            672         24  1.0              
sahirmaharajj/school-student-daily-attendance                   School Student Daily Attendance                    2MB  2024-04-29 19:29:56           4246         87  1.0              
supriyoain/water-quality-data                                   Water Quali

In [3]:
!unzip /content/brain-motor-imagery-classification.zip

Archive:  /content/brain-motor-imagery-classification.zip
  inflating: sample_submission.csv   
  inflating: test/00edaaa8-102d-4ee3-b531-0be67dcbd9e6.npy  
  inflating: test/01b63df8-05f9-4956-87f4-949ed9ae4dd6.npy  
  inflating: test/01bfd881-7850-49f1-ad32-ff9ae51e3e3f.npy  
  inflating: test/01d9002f-b086-4635-bf74-5316ceda3098.npy  
  inflating: test/028522a4-5332-4e84-b578-ce45ed5e4899.npy  
  inflating: test/0368a7cc-d6c6-416d-878e-edbcd562f317.npy  
  inflating: test/044841bc-050d-4c9f-92db-1ff1a7f306cb.npy  
  inflating: test/051581bf-c779-4d9c-86e9-324404b149ca.npy  
  inflating: test/06360991-5140-41a1-889f-3cb10b2241d0.npy  
  inflating: test/066dc317-8b3c-431e-b5d9-894abe3a4857.npy  
  inflating: test/06e9b7ab-0ee9-47a3-a500-a8dab4ee097f.npy  
  inflating: test/06f9ab25-f9b7-4a12-82f3-a3c92adcc645.npy  
  inflating: test/072dc9a9-7bfa-4da9-83d7-23b2ed5818e0.npy  
  inflating: test/075e9017-dace-441d-acbd-dd929c1a06f7.npy  
  inflating: test/0783e1ea-375e-4239-be7d-12177a54

<hr>

In [6]:
from scipy.signal import butter, filtfilt, iirnotch, freqz
import os
import numpy as np

In [65]:
# Band-pass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Notch filter
def notch_filter(data, cut_freq, fs, Q=30):
    nyq = 0.5 * fs
    freq = cut_freq / nyq
    b, a = iirnotch(freq, Q)
    y = filtfilt(b, a, data)
    return y

def filter03050(Ipsignal):
    sampling_rate = 250  # Hz
    # fft_signal = np.fft.fft(Ipsignal)
    # frequencies = np.fft.fftfreq(len(Ipsignal), 1/sampling_rate)
    filtered_signal = bandpass_filter(Ipsignal, 1, 30, sampling_rate)          # band-pass filter 0-30 Hz
    filtered_signal = notch_filter(filtered_signal, 50, sampling_rate)       # notch filter 50 Hz
    return filtered_signal

def unique_name(Datapath):
    all_name = os.listdir(Datapath)
    new = []
    for un in all_name:
        i = un.replace("_data_time_series.npy","").replace("_data_time_stamps.npy","").replace("_label_time_series.npy","").replace("_label_time_stamps.npy","")
        new.append(i)
        unique_name = list(set(new))
    return unique_name

def check_different_duration(label_time_stamps):
    keep_time = []
    start_time = 0
    for current_time in list(label_time_stamps):
        keep_time.append(current_time-start_time)
        start_time = current_time
    keep_time = keep_time[1:]
    avgtime = sum(keep_time)/len(keep_time)
    stat = True
    for etime in keep_time:
        if 6.8 <= etime <= 7.5:
            stat = True
        else:
            stat = False
            break
    return stat,avgtime

file_path = "/content/train/train"
unique_name = unique_name(file_path)

In [66]:
unique_name.remove("s1_d2_p002_003")
unique_name.remove("s1_d2_p006_008")
unique_name.remove("s2_d2_p002_006")

In [52]:
for un in unique_name:


s1_d2_p006_008
s2_d2_p006_004
s2_d2_p003_004
s1_d3_p019_002
s2_d2_p002_003
s1_d2_p004_005
s1_d2_p003_005
s2_d2_p006_005
s2_d3_p019_007
s2_d2_p007_008
s2_d2_p002_005
s1_d4_p020_006
s2_d3_p017_008
s2_d2_p010_001
s2_d2_p009_008
s2_d3_p015_007
s2_d3_p016_003
s1_d3_p013_003
s1_d3_p016_007
s1_d2_p009_004
s2_d3_p014_002
s1_d2_p011_001
s2_d2_p002_002
s1_d3_p013_007
s2_d2_p006_007
s2_d3_p017_003
s2_d2_p009_006
s2_d3_p014_004
s2_d2_p011_001
s2_d2_p009_001
s1_d2_p003_001
s2_d2_p006_002
s1_d2_p011_007
s2_d2_p010_008
s1_d3_p016_006
s1_d2_p008_001
s1_d2_p006_007
s2_d3_p017_001
s1_d3_p019_007
s2_d3_p019_003
s1_d4_p020_003
s1_d2_p003_006
s1_d3_p016_008
s2_d3_p016_002
s2_d3_p012_001
s1_d2_p002_005
s1_d2_p011_008
s1_d3_p016_005
s1_d3_p019_008
s2_d3_p016_004
s2_d3_p014_007
s1_d2_p002_002
s1_d2_p006_005
s1_d2_p010_006
s2_d3_p017_002
s2_d2_p010_005
s2_d3_p015_005
s1_d2_p004_007
s1_d2_p011_005
s2_d2_p007_002
s2_d3_p017_004
s1_d2_p003_007
s2_d3_p019_006
s2_d2_p002_007
s2_d2_p002_004
s2_d3_p019_008
s1_d2_p003

In [57]:
# sampling label != 30
for un in unique_name:
    label_time_series = np.load(os.path.join(file_path,un+"_label_time_series.npy"))
    if len(label_time_series) != 30:
        print(un)

s1_d2_p006_008
s2_d2_p002_006


In [74]:
data_dataset  = []
label_dataset = []
for name in unique_name:
    # print(name)
    signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals[:,:8]
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):
        continue
    # print("Avg time",self.avgtime)
    if filter!=None:
        filtered_signal = []
        Dim = signals.shape[1]
        # print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    sample_point = int(round(avgtime))
    datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        # start_point = index * datapoint
        # end_point = (index + 1) * datapoint
        # if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
        #     data_dataset.append(signals[start_point:end_point])
        # else:
        #     break
        sn = signals[1750*i:1750*(i+1)]
        if sn.shape[0]!=30 and sn.shape[1]!=8:
            print(name,"error")
        data_dataset.append(sn)
    label_dataset.append(label_time_series)

data_dataset = np.stack(data_dataset)

label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()
# print(data_dataset)
# print(label_dataset)
# data_sliced_with_label = tuple(zip(data_dataset,label_dataset))

In [75]:
data_dataset.shape

(5640, 1750, 8)

In [76]:
label_dataset = np.array(label_dataset)
label_dataset

array([120, 110, 150, ..., 150, 120, 110])

In [77]:
label_dataset.shape

(5640,)

<hr>

In [15]:
pip install braindecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [16]:
from braindecode.models.util import models_dict
print(f'All the Braindecode models:\n{list(models_dict.keys())}')

All the Braindecode models:
['ATCNet', 'Deep4Net', 'DeepSleepNet', 'EEGConformer', 'EEGITNet', 'EEGInception', 'EEGInceptionERP', 'EEGInceptionMI', 'EEGNetv1', 'EEGNetv4', 'EEGResNet', 'HybridNet', 'ShallowFBCSPNet', 'SleepStagerBlanco2020', 'SleepStagerChambon2018', 'SleepStagerEldele2021', 'TCN', 'TIDNet', 'USleep']


In [19]:
from braindecode.models import ShallowFBCSPNet              # Import the model from braindecode * There are many model that available
import mne
import numpy as np

In [18]:
model = ShallowFBCSPNet(
    n_chans   = 8,
    n_times   = 1750,
    n_outputs = 3,
    final_conv_length='auto',
)
print(model)

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 8, 1750]              [1, 3]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 8, 1750]              [1, 8, 1750, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 8, 1750, 1]           [1, 1, 1750, 8]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 8]           [1, 40, 1726, 1]          13,840                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1726, 1]          [1, 40, 1726, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1726, 1]          [1, 40, 111, 1]           --                        [75, 1]
├─Express

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [78]:
data_dataset.shape

(5640, 1750, 8)

In [79]:
label_dataset.shape

(5640,)

In [80]:
data_dataset = data_dataset.reshape(5640, 8, 1750)


In [81]:
info = mne.create_info(
    ch_names = ['c1','c2','c3','c4','c5','c6','c7','c8'], sfreq=250., ch_types="eeg"
)
# info = mne.create_info(ch_names=['C3', 'C4', 'Cz'], sfreq=256., ch_types='eeg')
# X = np.random.randn(100, 3, 1024)  # 100 epochs, 3 channels, 4 seconds (@256Hz)
epochs = mne.EpochsArray(data_dataset, info=info)
# y = np.random.randint(0, 4, size=100)  # 4 classes
# print(epochs)

Not setting metadata
5640 matching events found
No baseline correction applied
0 projection items activated


In [82]:
label_dataset = np.array(label_dataset)
label_dataset

array([120, 110, 150, ..., 150, 120, 110])

In [83]:
# Map value
mapping = {110: 0, 120: 1, 150: 2}

mapped_label = np.vectorize(mapping.get)(label_dataset)
print(mapped_label)

[1 0 2 ... 2 1 0]


In [84]:
label = np.array(mapped_label)
label

array([1, 0, 2, ..., 2, 1, 0])

In [85]:
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier

net = EEGClassifier(
    'ShallowFBCSPNet',
    module__final_conv_length='auto',
    train_split=ValidSplit(0.2),
    # To train a neural network you need validation split, here, we use 20%.
)
# Train model
net.fit(epochs, label)


/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1       53.7106       0.3200       22.6883  77.4733
      2        9.0016       0.3129        5.3519  77.4945
      3        3.9494       0.3103        2.7902  84.0351
      4        2.7364       0.3262        1.7301  76.7070
      5        2.1341       0.3307        1.5533  78.4001
      6        1.7510       0.2988        1.1987  77.9235
      7        1.6469       0.3147        1.4495  78.3469
      8        1.6153       0.2952        1.2819  79.6506
      9        1.5121       0.2970        1.4540  78.3526
     10        1.4657       0.2837        1.2744  83.3526


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 8, 1750]              [1, 3]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 8, 1750]              [1, 8, 1750, 1]           --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 8, 1750, 1]           [1, 1, 1750, 8]           --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 8]           [1, 40, 1726, 1]          13,840                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

<hr>

In [86]:
from scipy.signal import butter, filtfilt, iirnotch, freqz
import os

In [90]:
test_data = np.load("/content/Test_data/s1_d2_p003_001_data_time_series.npy")

In [91]:
data_dataset  = []
label_dataset = []
file_path = "/content/Test_data"
unique_name = ["s1_d2_p003_001"]
for name in unique_name:
    print(name)
    signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals[:,:8]
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):
        continue
    # print("Avg time",self.avgtime)
    if filter!=None:
        filtered_signal = []
        Dim = signals.shape[1]
        print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    sample_point = int(round(avgtime))
    datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        start_point = index * datapoint
        end_point = (index + 1) * datapoint
        if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
            data_dataset.append(signals[start_point:end_point])
        else:
            break
    label_dataset.append(label_time_series)

data_dataset = np.stack(data_dataset)

label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()

s1_d2_p003_001
8


In [92]:
data_dataset = data_dataset.reshape(30,8,1750)
data_dataset.shape

(30, 8, 1750)

In [93]:
test_epochs = mne.EpochsArray(data_dataset,info=info)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


In [94]:
# Get predictions from the trained model
predictions = net.predict(test_epochs)
probabilities = net.predict_proba(test_epochs)

In [96]:
label_time_series

array([[120],
       [110],
       [150],
       [150],
       [120],
       [110],
       [120],
       [110],
       [150],
       [150],
       [120],
       [110],
       [120],
       [150],
       [110],
       [150],
       [120],
       [110],
       [120],
       [110],
       [150],
       [150],
       [120],
       [110],
       [150],
       [120],
       [110],
       [150],
       [120],
       [110]], dtype=int32)

In [97]:
# Map value
mapping = {110: 0, 120: 1, 150: 2}

mapped_label = np.vectorize(mapping.get)(label_dataset)
print(mapped_label)

[1 0 2 2 1 0 1 0 2 2 1 0 1 2 0 2 1 0 1 0 2 2 1 0 2 1 0 2 1 0]


In [95]:
print("Predictions:", predictions)
print("Probabilities:", probabilities)

Predictions: [2 0 1 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 2 0 0 0 0 0 0 1 1]
Probabilities: [[0.39132205 0.20336382 0.40531412]
 [0.3653026  0.35641575 0.2782816 ]
 [0.3330554  0.40108722 0.26585737]
 [0.3324255  0.36565652 0.301918  ]
 [0.43043816 0.36224824 0.20731361]
 [0.3480283  0.38278493 0.26918674]
 [0.3680579  0.3848994  0.24704266]
 [0.3595839  0.378043   0.26237312]
 [0.40132222 0.3494912  0.24918655]
 [0.39405036 0.30345717 0.30249244]
 [0.32090598 0.4143984  0.2646956 ]
 [0.3703725  0.3646473  0.26498017]
 [0.32650247 0.37516588 0.29833165]
 [0.42864814 0.36864665 0.2027052 ]
 [0.38562804 0.35248932 0.26188257]
 [0.34257916 0.32655492 0.33086604]
 [0.48616192 0.28197917 0.23185885]
 [0.39772555 0.3384034  0.26387098]
 [0.41990045 0.30582514 0.27427447]
 [0.36511993 0.4139803  0.22089976]
 [0.34798858 0.3838852  0.2681262 ]
 [0.3554323  0.28561616 0.3589515 ]
 [0.43828323 0.30272058 0.2589962 ]
 [0.40020257 0.31807357 0.28172386]
 [0.36362913 0.3297255  0.30664533]
 [0.402532

In [99]:
for i,j in zip(mapped_label,predictions):
    print(i,j,i==j)

1 2 False
0 0 True
2 1 False
2 1 False
1 0 False
0 1 False
1 1 True
0 1 False
2 0 False
2 0 False
1 1 True
0 0 True
1 1 True
2 0 False
0 0 True
2 0 False
1 0 False
0 0 True
1 0 False
0 1 False
2 1 False
2 2 True
1 0 False
0 0 True
2 0 False
1 0 False
0 0 True
2 0 False
1 1 True
0 1 False


<hr>

# Observer Test

In [100]:
test01 = np.load("/content/test/00edaaa8-102d-4ee3-b531-0be67dcbd9e6.npy")

In [104]:
test01 = test01[:,:8]
test01.shape

(1750, 8)

In [105]:
# Reshape the array to (1, 8, 1750)
reshaped_test01 = test01.T.reshape(1, 8, 1750)

print(reshaped_test01.shape)

(1, 8, 1750)


In [107]:
test_epochs = mne.EpochsArray(reshaped_test01,info=info)

Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated


In [108]:
# Get predictions from the trained model
predictions = net.predict(test_epochs)
probabilities = net.predict_proba(test_epochs)

In [109]:
print("Predictions:", predictions)
print("Probabilities:", probabilities)

Predictions: [2]
Probabilities: [[2.8076928e-24 6.2339598e-18 1.0000000e+00]]
